# Procesamiento de pdfs

# columna id, partido politico,content

In [7]:
import os
import re
import pandas as pd
import pdfplumber

# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de diccionarios específicos a procesar
file_parameters = [
    {"file_name": "REVOLUCIÓN CIUDADANA - RETO _Plan de trabajo_.pdf"},
    {"file_name": "PARTIDO SOCIEDAD UNIDA MÁS ACCIÓN, SUMA _Plan de trabajo_.pdf"},
    {"file_name": "PARTIDO IZQUIERDA DEMOCRÁTICA _Plan de trabajo_.pdf"},
    {"file_name": "MOVIMIENTO CENTRO DEMOCRÁTICO _Plan de trabajo_.pdf"},
    {"file_name": "MOVIMIENTO CONSTRUYE _Plan de trabajo_.pdf"},
    {"file_name": "MOVIMIENTO CREO, CREANDO OPORTUNIDADES _Plan de trabajo_.pdf"},
    {"file_name": "MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPENDIENTE GENERANDO OPORTUNIDADES _Plan de trabajo_.pdf"},
    {"file_name": "MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA _PID_ _Plan de trabajo_.pdf"},
    {"file_name": "MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN _Plan de trabajo_.pdf"},
    {"file_name": "PARTIDO SOCIEDAD PATRIÓTICA  21 DE ENERO _Plan de trabajo_.pdf"},
    {"file_name": "PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf"},
    {"file_name": "PARTIDO SOCIALISTA ECUATORIANO _Plan de trabajo_.pdf"},
    {"file_name": "MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf"},
    {"file_name": "PARTIDO AVANZA _Plan de trabajo_.pdf"},
    {"file_name": "PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf"},
    {"file_name": "MOVIMIENTO DE UNIDAD PLURINACIONAL PACHAKUTIK _Plan de trabajo_.pdf"},
]

# Función para obtener el último ID del archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    df = pd.read_csv(csv_path, sep="|", encoding="utf-8")
    if df.empty:
        return 1
    return df['ID'].iloc[-1] + 1


# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Crear una lista para almacenar los datos
data = []

# Recorrer la lista de diccionarios específicos
for file_param in file_parameters:
    file_name = file_param["file_name"]

    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)

    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")

        # Agregar los datos a la lista
        data.append([file_id, processed_name])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Crear un DataFrame a partir de los datos nuevos
df_new = pd.DataFrame(data, columns=['ID', 'Nombre'])

# Verificar si el archivo CSV ya existe
if os.path.exists(output_csv):
    # Leer el archivo CSV existente
    df_existing = pd.read_csv(output_csv, sep="|", encoding="utf-8")
    # Concatenar los datos nuevos con los existentes
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
else:
    df_combined = df_new

# Guardar el DataFrame combinado en el archivo CSV con delimitador ";"
df_combined.to_csv(output_csv, sep=";", index=False, encoding="utf-8")

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


In [9]:
import os
import re
import pandas as pd
import pdfplumber

# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "oraciones.csv"

# Lista de diccionarios específicos a procesar
file_parameters = [
    {"file_name": "REVOLUCIÓN CIUDADANA - RETO _Plan de trabajo_.pdf", "exclude_pages_start": 8},
    {"file_name": "PARTIDO SOCIEDAD UNIDA MÁS ACCIÓN, SUMA _Plan de trabajo_.pdf", "exclude_pages_start": 7},
    {"file_name": "PARTIDO IZQUIERDA DEMOCRÁTICA _Plan de trabajo_.pdf","exclude_pages_start": 5},
    {"file_name": "MOVIMIENTO CENTRO DEMOCRÁTICO _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO CONSTRUYE _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO CREO, CREANDO OPORTUNIDADES _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPENDIENTE GENERANDO OPORTUNIDADES _Plan de trabajo_.pdf", "exclude_pages_start": 4},
    {"file_name": "MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA _PID_ _Plan de trabajo_.pdf", "exclude_pages_start": 3},
    {"file_name": "MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN _Plan de trabajo_.pdf", "exclude_pages_start": 3},
    {"file_name": "PARTIDO SOCIEDAD PATRIÓTICA  21 DE ENERO _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO SOCIALISTA ECUATORIANO _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO AVANZA _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf", "exclude_pages_start": 2},
    {"file_name": "MOVIMIENTO DE UNIDAD PLURINACIONAL PACHAKUTIK _Plan de trabajo_.pdf", "exclude_pages_start": 1}
]

# Función para obtener el último ID del archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    df = pd.read_csv(csv_path, sep="|", encoding="utf-8")
    if df.empty:
        return 1
    return df['ID'].iloc[-1] + 1

# Función para extraer texto del PDF excluyendo las primeras y últimas páginas
def extract_text_excluding_pages(pdf_path, exclude_pages_start, exclude_pages_end=1):
    extracted_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(exclude_pages_start, len(pdf.pages) - exclude_pages_end):
            page_text = pdf.pages[i].extract_text()
            if page_text:
                extracted_text += page_text + "\n"
    return extracted_text.strip()

# Función para limpiar el contenido del texto
def clean_content(text):
    # Eliminar viñetas comunes
    text = re.sub(r"[\u2022\u25CB\u2023\u2219\u2022\u25AA\u25B6\u25B7\u25C6\u2043\u25B8\u25BB\u2660\u25FE\u25FB]", "", text)
    text = re.sub(r'\(cid:\d+\)', '', text)
    # Eliminar enumeraciones (números seguidos de punto)
    text = re.sub(r'^\d+\.', '', text)  # Al inicio de la línea
    text = re.sub(r'\n\d+\.', '\n', text)  # En medio del texto
    
    # Reemplazar múltiples espacios con uno solo
    text = re.sub(r'\s+', ' ', text)
    
    # Eliminar espacios al inicio y final
    text = text.strip()
    
    return text

# Función para dividir el texto en oraciones
def dividir_oraciones_por_id(text, text_id):
    delimitadores = '.'
    oraciones = []
    oracion_actual = ""
    for char in text:
        oracion_actual += char
        if char in delimitadores:
            oraciones.append(oracion_actual.strip())
            oracion_actual = ""
    if oracion_actual:  # Si hay algo restante
        oraciones.append(oracion_actual.strip())
    
    # Crear una lista de tuplas con id y oraciones
    return [(text_id, i, oracion) for i, oracion in enumerate(oraciones, start=1)]

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Crear una lista para almacenar los datos
data = []

# Recorrer la lista de diccionarios específicos
for file_param in file_parameters:
    file_name = file_param["file_name"]
    exclude_pages_start = file_param["exclude_pages_start"]

    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)

    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")

        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path, exclude_pages_start=exclude_pages_start)

        # Limpiar el contenido extraído
        cleaned_content = clean_content(content)

        # Dividir el contenido en oraciones
        oraciones = dividir_oraciones_por_id(cleaned_content, file_id)

        # Agregar las oraciones a la lista de datos
        data.extend(oraciones)

        # Incrementar el ID
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Crear un DataFrame a partir de las oraciones
df_oraciones = pd.DataFrame(data, columns=['ID', 'Oracion_ID', 'Oracion'])

# Verificar si el archivo CSV ya existe
if os.path.exists(output_csv):
    # Leer el archivo CSV existente
    df_existing = pd.read_csv(output_csv, sep="|", encoding="utf-8")
    # Concatenar los datos nuevos con los existentes
    df_combined = pd.concat([df_existing, df_oraciones], ignore_index=True)
else:
    df_combined = df_oraciones

# Guardar el DataFrame combinado en el archivo CSV con delimitador ";"
df_combined.to_csv(output_csv, sep=";", index=False, encoding="utf-8")

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: oraciones.csv


# ocr

In [11]:
import pytesseract
from pdf2image import convert_from_path
import csv
import os
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd

# Descargar el recurso necesario para tokenizar oraciones
nltk.download('punkt')

csv.field_size_limit(1000000)

# Configuración global
pdf_directory = "./data/"
csv_file = "oraciones.csv"
columns = ['ID', 'Nombre', 'Contenido']

# Configurar Tesseract para Fedora
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

def procesar_pdf(ruta_pdf, id_asignado, nombre_doc):
    try:
        # Convertir PDF a imágenes
        images = convert_from_path(ruta_pdf, dpi=300)
        
        # Extraer y limpiar texto
        contenido = " ".join(
            [pytesseract.image_to_string(img, lang='spa').strip().replace('\n', ' ') 
             for img in images]
        )
        
        # Tokenizar el texto en oraciones
        oraciones = sent_tokenize(contenido, language='spanish')
        
        # Asignar ID único a cada oración
        oraciones_ids = []
        oraciones_texto = []
        
        for i, oracion in enumerate(oraciones):
            oraciones_ids.append(f"{id_asignado}_{i}")  # ID único para cada oración
            oraciones_texto.append(oracion)  # Texto de la oración
        
        # Crear DataFrame con solo los campos requeridos
        data = {
            'ID': [id_asignado] * len(oraciones),
            'Oracion_ID': oraciones_ids,
            'Oracion': oraciones_texto
        }
        df_oraciones = pd.DataFrame(data, columns=['ID', 'Oracion_ID', 'Oracion'])
        
        # Escribir el DataFrame al CSV (o concatenar al existente)
        if os.path.exists(csv_file):
            df_oraciones.to_csv(csv_file, mode='a', header=False, index=False, sep=';')
        else:
            df_oraciones.to_csv(csv_file, mode='w', header=True, index=False, sep=';')
        
        return True
    except Exception as e:
        print(f"Error procesando {ruta_pdf}: {str(e)}")
        return False

# Mapeo de archivos a IDs y nombres
documentos = {
    "PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf": {"id": 11, "nombre": "PARTIDO UNIDAD POPULAR"},
    "MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf": {"id": 13, "nombre": "MOVIMIENTO DEMOCRACIA SÍ"},
    "PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf": {"id": 15, "nombre": "PARTIDO SOCIAL CRISTIANO"}   
}

# Procesar todos los documentos
for archivo, datos in documentos.items():
    ruta_completa = os.path.join(pdf_directory, archivo)
    if os.path.exists(ruta_completa):
        if procesar_pdf(ruta_completa, datos['id'], datos['nombre']):
            print(f"{archivo} procesado (ID {datos['id']})")
    else:
        print(f"Archivo no encontrado: {ruta_completa}")

print("\nProceso completado.")


[nltk_data] Downloading package punkt to /home/alech/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf procesado (ID 11)
MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf procesado (ID 13)
PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf procesado (ID 15)

Proceso completado.


# limpiar la columna oracion

In [ ]:
import csv
import re
import os

# Función para limpiar el contenido del texto
def clean_content(text):
    text = text.lower()
    # Eliminar viñetas comunes
    text = re.sub(r"[\u2022\u25CB\u2023\u2219\u2022\u25AA\u25B6\u25B7\u25C6\u2043\u25B8\u25BB\u2660\u25FE\u25FB]", "", text)
    
    # Eliminar (cid:...) - Referencias CID
    text = re.sub(r'\(cid:\d+\)', '', text)
    
    # Eliminar enumeraciones (números seguidos de punto)
    text = re.sub(r'^\d+\.', '', text)  # Al inicio de la línea
    text = re.sub(r'\n\d+\.', '\n', text)  # En medio del texto

    # Eliminar la enumeración de página (ejemplo: 'Página 1', 'pág. 2', etc.)
    text = re.sub(r'Página \d+', '', text)
    text = re.sub(r'pág\.\s*\d+', '', text)
    text = re.sub(r'pag\.\s*\d+', '', text)
    text = re.sub(r'Page \d+', '', text)
    text = re.sub(r'page \d+', '', text)

    # Eliminar caracteres especiales no alfabéticos ni numéricos (como @, #, $, etc.)
    text = re.sub(r'[^\w\s]', '', text)

    # Reemplazar múltiples espacios con uno solo
    text = re.sub(r'\s+', ' ', text)
    
    # Eliminar espacios al inicio y final
    text = text.strip()
    
    return text


# Leer el archivo CSV, limpiar el contenido de la columna "Oracion", ordenar por ID y guardar
def limpiar_y_guardar_csv(csv_file):
    try:
        filas_existentes = []
        
        # Leer las filas existentes desde el CSV
        if os.path.exists(csv_file):
            with open(csv_file, 'r', encoding='utf-8-sig') as f:
                reader = csv.DictReader(f, delimiter=';')
                for row in reader:
                    # Limpiar el contenido de la columna "Oracion"
                    row['Oracion'] = clean_content(row['Oracion'])
                    
                    # Verificar si la columna "Oracion" no está vacía
                    if row['Oracion']:  
                        filas_existentes.append(row)
        
        # Ordenar las filas por ID (conversión a int para evitar errores de ordenación)
        filas_existentes.sort(key=lambda x: int(x['ID']))

        # Escribir las filas modificadas en el archivo CSV
        with open(csv_file, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=['ID', 'Oracion_ID', 'Oracion'], delimiter=';')
            writer.writeheader()
            writer.writerows(filas_existentes)
        
        print(f"Archivo {csv_file} procesado, limpiado y ordenado correctamente.")
    
    except Exception as e:
        print(f"Error procesando el archivo CSV: {str(e)}")

# Llamar a la función
limpiar_y_guardar_csv('oraciones.csv')


✅ Archivo oraciones.csv procesado, limpiado y ordenado correctamente.


In [1]:
import spacy
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd

# Descargar recursos necesarios de NLTK
nltk.download("punkt")

# Cargar modelo de lenguaje en español
nlp = spacy.load("es_core_news_sm")

# Cargar el CSV con las oraciones
df = pd.read_csv("oraciones.csv", sep=";")

# Lista ampliada de temas clave
temas_relevantes = {
    "economía", "educación", "salud", "seguridad", "empleo",
    "infraestructura", "corrupción", "tecnología", "ambiente",
    "justicia", "transporte", "política", "desarrollo", "energía",
    "derechos humanos", "igualdad", "innovación", "turismo",
    "agricultura", "cultura", "deporte", "finanzas", "inversión",
    "vivienda", "servicios públicos", "ciencia", "medio ambiente",
    "gobierno", "industria", "exportaciones", "importaciones",
    "educación superior", "sanidad", "movilidad", "inteligencia artificial",
    "seguridad ciudadana", "crimen organizado", "democracia", "pobreza",
    "sostenibilidad", "digitalización", "gestión pública", "comercio",
    "cambio climático"
}

# Función para extraer propuestas y detectar temas clave
def extraer_propuestas_y_temas(texto):
    if pd.isna(texto):  # Manejar valores nulos
        return "", ""

    oraciones = sent_tokenize(texto, language="spanish")  # Dividir en oraciones
    propuestas = []
    temas_detectados = set()

    for oracion in oraciones:
        doc = nlp(oracion)  
        
        # Detectar oraciones con verbos de acción
        if any(token.pos_ == "VERB" for token in doc):
            propuestas.append(oracion)

            # Identificar temas clave dentro de la oración
            temas_detectados.update({tema for tema in temas_relevantes if tema in oracion.lower()})

    # Tomar hasta 3 propuestas clave
    return "; ".join(propuestas[:3]), ", ".join(temas_detectados)

# Aplicar la extracción en cada fila del DataFrame
df[["Propuestas", "Temas Clave"]] = df["Oracion"].apply(lambda txt: pd.Series(extraer_propuestas_y_temas(txt)))

# Guardar el nuevo CSV con las nuevas columnas sin eliminar datos anteriores
df.to_csv("oraciones_actualizado.csv", index=False, sep=";")

print("✅ CSV actualizado con éxito. Se agregaron las columnas 'Propuestas' y 'Temas Clave'.")


ModuleNotFoundError: No module named 'spacy'

# Stop words, tokenizar,stemming

In [15]:
import nltk
import csv
import re
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer  # Usamos stemmer para español

# Descargar recursos necesarios
nltk.download('punkt')
nltk.download('stopwords')

# Inicializar el stemmer en español
stemmer = SnowballStemmer('spanish')

# Función para tokenizar, eliminar stopwords y aplicar stemming
def preprocesar_texto(texto):
    # Tokenización
    tokens = word_tokenize(texto.lower(), language='spanish')
    
    # Eliminar stopwords y caracteres no alfabéticos
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens 
              if word.isalpha() 
              and word not in stop_words
              and len(word) > 2]
    
    # Stemming
    tokens = [stemmer.stem(word) for word in tokens]
    
    return ' '.join(tokens)

# Leer el archivo CSV, limpiar el contenido de la columna "Oracion", eliminar filas vacías y luego escribir las filas nuevamente
def limpiar_y_guardar_csv(csv_file, output_csv):
    try:
        filas_existentes = []
        
        # Leer las filas existentes desde el CSV
        if os.path.exists(csv_file):
            with open(csv_file, 'r', encoding='utf-8-sig') as f:
                reader = csv.DictReader(f, delimiter=';')
                for row in reader:
                    # Limpiar el contenido de la columna "Oracion"
                    if 'Oracion' in row:
                        row['Oracion'] = clean_content(row['Oracion'])
                        # Procesar el contenido: tokenizar, eliminar stopwords y aplicar stemming
                        row['Oracion'] = preprocesar_texto(row['Oracion'])
                    
                    # Verificar si la columna "Oracion" no está vacía
                    if row['Oracion']:  # Si no está vacío o solo contiene espacios
                        filas_existentes.append(row)
        
        # Escribir las filas modificadas en el archivo CSV
        with open(output_csv, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=['ID', 'Oracion_ID', 'Oracion'], delimiter=';')
            writer.writeheader()
            writer.writerows(filas_existentes)
        
        print(f"Archivo procesado y guardado correctamente en: {output_csv}")
    
    except Exception as e:
        print(f"Error procesando el archivo CSV: {str(e)}")

# Llamar a la función
input_csv = 'oraciones.csv'
output_csv = 'oraciones_procesadas.csv'
limpiar_y_guardar_csv(input_csv, output_csv)


[nltk_data] Downloading package punkt to /home/alech/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alech/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Archivo procesado y guardado correctamente en: oraciones_procesadas.csv


# enbeddings Bert y faiss

In [23]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
import faiss
import pickle

# Verificar si hay GPU disponible
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Cargar el archivo CSV
df = pd.read_csv('oraciones_procesadas.csv', delimiter=';')  # Cambia el nombre de tu archivo CSV si es necesario

# Inicializar el modelo BERT preentrenado y moverlo a la GPU si está disponible
model = SentenceTransformer('paraphrase-MiniLM-L6-v2', device=device)

# Crear embeddings para todas las oraciones
embeddings = model.encode(df['Oracion'].tolist(), show_progress_bar=True, device=device)

# Convertir los embeddings en un formato compatible con FAISS (float32)
embeddings = np.array(embeddings).astype('float32')

# Crear un índice FAISS
dimension = embeddings.shape[1]  # Dimensión de los embeddings
index = faiss.IndexFlatL2(dimension)  # Índice basado en L2 (distancia euclidiana)
index.add(embeddings)  # Agregar los embeddings al índice FAISS

# Guardar los embeddings y el índice FAISS en archivos
np.save('embeddings.npy', embeddings)  # Guardamos los embeddings en un archivo .npy
faiss.write_index(index, 'faiss_index.index')  # Guardamos el índice FAISS en un archivo .index

# Guardar los ID y las oraciones en un archivo .pkl para cargarlos fácilmente después
with open('sentences.pkl', 'wb') as f:
    pickle.dump(df[['ID', 'Oracion']].to_dict(orient='records'), f)

print("Embeddings y FAISS guardados exitosamente.")


Batches:   0%|          | 0/421 [00:00<?, ?it/s]

Embeddings y FAISS guardados exitosamente.


In [32]:
import pandas as pd
import faiss
import numpy as np
import pickle
import nltk
from nltk.stem import SnowballStemmer

# Descargar recursos de NLTK si es necesario
nltk.download('punkt')

# Inicializar el stemmer en español
stemmer = SnowballStemmer("spanish")

# Cargar los embeddings y el índice FAISS
embeddings = np.load('embeddings.npy')  
index = faiss.read_index('faiss_index.index')  

# Cargar las oraciones desde el archivo .pkl
with open('sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

# Cargar el archivo CSV con los candidatos
df_candidatos = pd.read_csv('candidatos.csv', sep=';')  # Asegúrate de que el separador sea correcto

# Función para aplicar stemming a un texto
def stem_text(text):
    words = nltk.word_tokenize(text, language="spanish")  # Tokenizar el texto
    words_stemmed = [stemmer.stem(word) for word in words]  # Aplicar stemming
    return " ".join(words_stemmed)  # Reunir las palabras nuevamente en una oración

# Función para buscar oraciones que contienen términos similares a la consulta
def buscar_por_consulta(query):
    query_stemmed = stem_text(query)  # Aplicar stemming a la consulta

    # Filtrar las oraciones que contengan términos relacionados con la consulta procesada
    oraciones_filtradas = [
        oracion for oracion in sentences if query_stemmed in stem_text(oracion['Oracion'])
    ]

    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(oraciones_filtradas)

    # Si hay resultados, buscamos los partidos
    if not df_resultados.empty:
        df_resultados['Partido'] = df_resultados['ID'].map(lambda id_: obtener_partido(id_))
        print(df_resultados[['ID', 'Oracion', 'Partido']])
    else:
        print("No se encontraron oraciones que coincidan con la consulta.")

# Función para buscar el partido según el ID en `candidatos.csv`
def obtener_partido(id_):
    partido = df_candidatos.loc[df_candidatos['ID'] == id_, 'Partido']
    return partido.values[0] if not partido.empty else "No encontrado"

# Ejemplo de consulta:
query = "escuelas"
buscar_por_consulta(query)


[nltk_data] Downloading package punkt to /home/alech/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ID                                            Oracion  \
0     1  recuper gratuid prepar ingres escuel polic nac...   
1     1  promov prevencion violenci muj edad tempran tr...   
2     1  establec mecan inmigr edad escol menor años as...   
3     1  foment educ artist particip joven activ cultur...   
4     2  construccion hospital escuel carreter provinci...   
..   ..                                                ...   
109  11  cumpl plan nacional reinsercion estudiantil mi...   
110  11  contrat especial salud mental dentr escuel col...   
111  11  promov profesionaliz artist popular vincul doc...   
112  11  mism polit soberan concesion red fortalec empr...   
113  15  signif sol cubr demand dej estudi pued acced s...   

                                      Partido  
0                REVOLUCIÓN CIUDADANA - RETO   
1                REVOLUCIÓN CIUDADANA - RETO   
2                REVOLUCIÓN CIUDADANA - RETO   
3                REVOLUCIÓN CIUDADANA - RETO   
4    PARTIDO SO

Ollama

In [6]:
import pandas as pd
import faiss
import numpy as np
import pickle
import nltk
import ollama
from nltk.stem import SnowballStemmer

# Descargar recursos de NLTK si es necesario
nltk.download('punkt')

# Inicializar el stemmer en español
stemmer = SnowballStemmer("spanish")

# Cargar los embeddings y el índice FAISS
embeddings = np.load('embeddings.npy')  
index = faiss.read_index('faiss_index.index')  

# Cargar las oraciones desde el archivo .pkl
with open('sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

# Cargar el archivo CSV con los candidatos
df_candidatos = pd.read_csv('candidatos.csv', sep=';')  # Asegúrate de que el separador sea correcto

# Función para aplicar stemming a un texto
def stem_text(text):
    words = nltk.word_tokenize(text, language="spanish")  # Tokenizar el texto
    words_stemmed = [stemmer.stem(word) for word in words]  # Aplicar stemming
    return " ".join(words_stemmed)  # Reunir las palabras nuevamente en una oración

# Función para buscar el partido según el ID en candidatos.csv
def obtener_partido(id_):
    partido = df_candidatos.loc[df_candidatos['ID'] == id_, 'Partido']
    return partido.values[0] if not partido.empty else "No encontrado"

# Función para buscar oraciones que contienen términos similares a la consulta
def buscar_por_consulta(query):
    query_stemmed = stem_text(query)  # Aplicar stemming a la consulta

    # Filtrar las oraciones que contengan términos relacionados con la consulta procesada
    oraciones_filtradas = [
        oracion for oracion in sentences if query_stemmed in stem_text(oracion['Oracion'])
    ]

    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(oraciones_filtradas)

    # Si hay resultados, agregamos el partido
    if not df_resultados.empty:
        df_resultados['Partido'] = df_resultados['ID'].map(lambda id_: obtener_partido(id_))
        return df_resultados[['ID', 'Oracion', 'Partido']]
    else:
        return None

# Función para generar respuesta en lenguaje natural con Ollama
def generar_respuesta(query):
    resultados = buscar_por_consulta(query)
    
    if resultados is not None:
        # Convertimos los resultados en un texto para Ollama
        texto_resultados = "\n".join([f"ID: {row['ID']}, Oración: {row['Oracion']}, Partido: {row['Partido']}" 
                                      for _, row in resultados.iterrows()])

        # Generamos la respuesta con Ollama
        prompt = f"""
        Basado en la consulta '{query}', encontré estas oraciones y sus respectivos partidos:
        
        {texto_resultados}

        Responde en lenguaje natural resumiendo la información de manera clara y comprensible.
        """
        respuesta = ollama.chat(model='deepseek-r1:14b', messages=[{'role': 'user', 'content': prompt}])

        return respuesta['message']['content']
    else:
        return "No se encontraron resultados para tu consulta."

# Ejemplo de uso
query = "economia"
respuesta_final = generar_respuesta(query)
print(respuesta_final)

[nltk_data] Downloading package punkt to /home/alech/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<think>
Alright, so I'm trying to help someone with their query about the economy and political parties. They provided a list of IDs, sentences, and corresponding parties. The task is to summarize this information in a clear and understandable way.

First, I'll need to read through each entry carefully. It looks like each ID has multiple oraciones (sentences) related to economic topics, each tied to a specific partido político (political party). My goal is to group these sentences by their respective parties and then explain what each party is focusing on economically.

Starting with ID 2, all the sentences are from "PARTIDO SOCIEDAD UNIDA MÁS ACCIÓN, SUMA." The topics here seem to be about economic development, maybe creating special zones or boards for economic growth, similar to Singapore's model. So I'll note that this party is talking about economic development and special zones.

Next, ID 4 is from "MOVIMIENTO CENTRO DEMOCRÁTICO." The sentences mention things like economic-social

In [8]:
import pandas as pd
import faiss
import numpy as np
import pickle
import nltk
import ollama
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Descargar recursos de NLTK si es necesario
nltk.download('punkt')
nltk.download('stopwords')

# Inicializar el stemmer en español y cargar stopwords
stemmer = SnowballStemmer("spanish")
stop_words = set(stopwords.words("spanish"))

# Cargar los embeddings y el índice FAISS
embeddings = np.load('embeddings.npy')  
index = faiss.read_index('faiss_index.index')  

# Cargar las oraciones desde el archivo .pkl
with open('sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

# Cargar el archivo CSV con los candidatos
df_candidatos = pd.read_csv('candidatos.csv', sep=';')  # Asegúrate de que el separador sea correcto

# Función para preprocesar el texto: tokenización, eliminación de stopwords y stemming
def preprocess_text(text):
    words = word_tokenize(text, language="spanish")  # Tokenizar el texto
    words_filtered = [word for word in words if word.isalnum() and word.lower() not in stop_words]  # Eliminar stopwords
    words_stemmed = [stemmer.stem(word) for word in words_filtered]  # Aplicar stemming
    return " ".join(words_stemmed)  # Reunir las palabras nuevamente en una oración

# Función para obtener el partido según el ID en candidatos.csv
def obtener_partido(id_):
    partido = df_candidatos.loc[df_candidatos['ID'] == id_, 'Partido']
    return partido.values[0] if not partido.empty else "No encontrado"

# Función para buscar oraciones que contienen términos similares a la consulta
def buscar_por_consulta(query):
    query_processed = preprocess_text(query)  # Preprocesar la consulta del usuario

    # Filtrar oraciones con términos relacionados con la consulta procesada
    oraciones_filtradas = [
        oracion for oracion in sentences if query_processed in preprocess_text(oracion['Oracion'])
    ]

    # Crear un DataFrame con los resultados
    df_resultados = pd.DataFrame(oraciones_filtradas)

    # Si hay resultados, agregar la columna de partidos
    if not df_resultados.empty:
        df_resultados['Partido'] = df_resultados['ID'].map(lambda id_: obtener_partido(id_))
        return df_resultados[['ID', 'Oracion', 'Partido']]
    else:
        return None

# Función para generar respuesta en lenguaje natural con Ollama
def generar_respuesta(query):
    resultados = buscar_por_consulta(query)
    
    if resultados is not None:
        # Convertimos los resultados en un texto para Ollama
        texto_resultados = "\n".join([f"ID: {row['ID']}, Oración: {row['Oracion']}, Partido: {row['Partido']}" 
                                      for _, row in resultados.iterrows()])

        # Generamos la respuesta con Ollama
        prompt = f"""
        Basado en '{query}', encontré la siguiente información relevante:

        {texto_resultados}

        dame la lista entera de canditados con esa informacion
        """
        respuesta = ollama.chat(model='deepseek-r1:14b', messages=[{'role': 'user', 'content': prompt}])

        return respuesta['message']['content']
    else:
        return "No se encontraron resultados para tu consulta."

# Ejemplo de uso
query = "¿Cómo afecta la economía al país?"
respuesta_final = generar_respuesta(query)
print(respuesta_final)


[nltk_data] Downloading package punkt to /home/alech/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alech/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<think>
Bueno, el usuario me ha proporcionado una información relevante que extrajo de la pregunta "¿Cómo afecta la economía al país?". La información incluye un ID (2), una oración en español, y menciona a un partido político llamado "PARTIDO SOCIEDAD UNIDA MÁS ACCIÓN, SUMA".

Primero, entiendo que el usuario quiere una lista completa de candidatos con esa misma información. Sin embargo, solo ha proporcionado un ejemplo con ID 2 y ORACIÓN, así como el nombre del partido.

Para generar una respuesta adecuada, necesito asumir que hay más candidatos y posiblemente más IDs y oraciones relacionadas con la economía y su impacto en el país. Imagino que cada candidato podría tener un ID único, una oración relevante y pertenecer a un partido político.

Después de realizar esta asunción, procedo a crear una lista de candidatos imaginaria, asignando IDs consecutivos (desde 1 en adelante), creando oraciones relevantes que conectan la economía con el país, y atribuyendo cada uno al partido "PARTID